### [POC 3]  NGLY1 deficiency patient extraction

This analysis attempts to extract as many details as possible about NGLY1 patients from 2 full-text papers on the subject.  Outline:

1. Define a minimal prompt to per-study patient identifiers and free-text descriptions of associated information
2. Summarize the results for each patient, across studies, in a standard schema
3. Analyze the patients

In [2]:
%load_ext autoreload
%autoreload 2
import io
import sys
import pandas as pd
import matplotlib.pyplot as plt
from ngly1_gpt import utils, llm, doc
import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)
pd.set_option("display.max_colwidth", None, "display.max_rows", 400, "display.max_columns", None)

In [3]:
prompt = (utils.get_paths().prompts / "patient_extraction_1.txt").read_text()
print(prompt)

Text will be provided that contains information from a published, biomedical research article about {disease}.  Extract details about the patients discussed in this text. 

Requirements:

- Exclude any patients where context dictates that they do NOT have {disease}, e.g. when {disease} patients are compared to similar patients with other diseases.
- Extract as much information as possible about each patient including associated genotypes, phenotypes, physical or behavioral traits, demographics, lab measurements, treatments, family histories or anything else of clinical and/or biological relevance.
- Extract this information in CSV format with the following headers:
  - `patient_id`: Identifying information for the patient within the context of the article; typically an integer or anonymized id like "Patient 1". If some information applies to ALL patients in a study and the context does not make it possible to enumerate the patient ids, report only the value "ALL"
  - `external_study`: 

##### Execution

The prompt was run for all paper chunks via a command like:

```bash
PYTHONPATH="$(pwd)" python ngly1_gpt/cli.py extract_patients --output-filename=patients_1.tsv 2>&1 | tee data/logs/extract_patients_1.log.txt
```

The logs for these extractions showing all prompts and results are in [data/logs](data/logs).

#### Examples

#### Cleanup

In [14]:
patients_raw = pd.read_csv(utils.get_paths().output_data / "patients_1.tsv", sep="\t")
patients_raw.info()
patients_raw.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   patient_id                         173 non-null    object
 1   external_study                     14 non-null     object
 2   details                            173 non-null    object
 3   doc_id                             180 non-null    object
 4   doc_filename                       180 non-null    object
 5   patient_id,external_study,details  7 non-null      object
dtypes: object(6)
memory usage: 8.6+ KB


,patient_id,external_study,details,doc_id,doc_filename,"patient_id,external_study,details"
0,ALL,NaN,"All patients had global developmental delay, a movement disorder, and hypotonia.",PMC4243708,PMC4243708.txt,NaN
1,SOME,NaN,7 out of 8 patients had hypo- or alacrima.,PMC4243708,PMC4243708.txt,NaN
2,SOME,NaN,6 out of 7 patients had elevated liver transaminases.,PMC4243708,PMC4243708.txt,NaN
3,SOME,NaN,6 out of 8 patients had microcephaly.,PMC4243708,PMC4243708.txt,NaN
4,SOME,NaN,6 out of 8 patients had diminished reflexes.,PMC4243708,PMC4243708.txt,NaN
5,SOME,NaN,5 out of 6 patients had hepatocyte cytoplasmic storage material or vacuolization.,PMC4243708,PMC4243708.txt,NaN
6,SOME,NaN,4 out of 8 patients had seizures.,PMC4243708,PMC4243708.txt,NaN
7,ALL,NaN,The most common deleterious allele was the nonsense mutation c.1201A>T (p.R401X).,PMC4243708,PMC4243708.txt,NaN
8,ALL,NaN,"NGLY1 deficiency is a novel autosomal recessive disorder of the ERAD pathway associated with neurological dysfunction, abnormal tear production, and liver disease. The majority of patients detected to date carry a specific nonsense mutation that appears to be associated with severe disease.",PMC4243708,PMC4243708.txt,NaN
9,Patient 1,Need et al.,"3-year-old boy with compound heterozygous inactivating mutations in NGLY1. Clinical phenotype suggestive of a congenital disorder of glycosylation, although repeated transferrin isoelectric focusing and N-glycan analyses were normal. Liver biopsy showed accumulation of an amorphous unidentified substance throughout the cytoplasm, a finding likely consistent with NGLY1 dysfunction, which would be expected to result in abnormal accumulation of misfolded glycoproteins because of impaired cytosolic degradation.",PMC4243708,PMC4243708.txt,NaN


In [15]:
patients_raw.dropna(subset='patient_id,external_study,details')

,patient_id,external_study,details,doc_id,doc_filename,"patient_id,external_study,details"
62,NaN,NaN,NaN,PMC4243708,PMC4243708.txt,"ALL,24,Patients with autosomal recessive mutations in ERLIN2 present profound intellectual disability, developmental regression and multiple contractures. Despite the severity of the intellectual disability and neuromuscular findings, the results of brain imaging, electromyography and muscle biopsy appeared normal in the initial erlin2-deficient patients."
63,NaN,NaN,NaN,PMC4243708,PMC4243708.txt,"ALL,26,Another family was found to have a homozygous null mutation in ERLIN2, with affected individuals presenting with a hereditary spastic paraplegia phenotype."
83,NaN,NaN,NaN,PMC7477955,PMC7477955.txt,"ALL,NA,Clinical studies were designed to detail the phenotypic features of NGLY1-CDDG. Blood, urine, cerebral spinal fluid (CSF), lymphoblasts, and primary dermal fibroblasts were collected, analyzed, and stored. Studies included brain magnetic resonance imaging and spectroscopy (MRI and MRS, supplementary methods), routine and overnight electroencephalograms (EEGs) with a limited montage performed during a sleep study, electromyogram (EMG, supplementary methods) and nerve conduction studies (NCS, supplementary methods), indirect calorimetry, awake and sedated eye examination with Schirmer II testing, optical coherence tomography scans and electroretinography, behavioral determination of pure tone thresholds, tympanometry, distortion product otoacoustic emissions, auditory brainstem evoked potentials (ABR), quantitative sweat analysis autonomic testing (QSWEAT, supplementary methods), gastric aspiration, swallow study, skeletal survey, bone age, dual X-ray absorptiometry (DEXA), abdominal ultrasound, vibration controlled transient elastography (Fibroscan)12, echocardiogram, and electrocardiogram. Consultations included clinical neurology, audiology, nutrition, ophthalmology, hepatology, growth, puberty and hormonal studies, allergy and immunology, genetic counseling, physiatry, and speech, occupational, and physical therapy."
84,NaN,NaN,NaN,PMC7477955,PMC7477955.txt,"ALL,NA,Eleven individuals underwent developmental psychological evaluations, consisting of at least the Vineland Adaptive Behavior Scales, 2nd edition. Cognitive function was assessed with testing specific for age and developmental level that provided either an intelligence quotient (IQ) or developmental quotient (DQ) score."
85,NaN,NaN,NaN,PMC7477955,PMC7477955.txt,"ALL,NA,The Nijmegen pediatric CDG rating scale, a measure of clinical disease progression developed for CDG, was applied to all affected individuals younger than 18 years."
131,NaN,NaN,NaN,PMC7477955,PMC7477955.txt,"ALL,NA,Compared to the reference population, N-acetylaspartylglutamate + N-acetylaspartate (NAA) was lower than normal in the left centrum semiovale (LCSO) (p=0.004), the midline parietal grey matter (PGM) (p=0.02), and superior cerebellar vermis (SVERM) (p<0.0001). There was a deficit of glutamine + glutamate + gamma-aminobutyric acid (Glx) in the PGM (p=0.03), LCSO (p=0.01), and pons (p=0.0002). Choline was higher than expected for age only in the LCSO (p=0.0097), and myo-inositol was higher than expected for age in the pons (p=0.002). Multiple correlations between these MRS-measured metabolites and age, functional assessments, brain volume, and neurotransmitters in the CSF were found. The general trend showed that the differences noted above became more pronounced with increasing age, worsening function, and lower brain volume. MRS metabolite measurements did not correlate with total CSF protein, CSF albumin, or CSF/serum albumin ratio. There was a weak correlation (p=0.09) between atrophy and total CSF protein, but not CSF albumin or CSF/serum albumin ratio."
169,NaN,NaN,NaN,PMC7477955,PMC7477955.txt,"ALL,NA,Strong correlation between brain atrophy on MRI and functional assessments suggests that loss of neurons contributes to the functional impairment. The atr

In [19]:
patients_patch = (
    patients_raw.dropna(subset='patient_id,external_study,details')
    ['patient_id,external_study,details'].str.split(',', n=2, expand=True)
    .rename(columns={0: 'patient_id', 1: 'external_study', 2: 'details'})
)
patients_patch

,patient_id,external_study,details
62,ALL,24,"Patients with autosomal recessive mutations in ERLIN2 present profound intellectual disability, developmental regression and multiple contractures. Despite the severity of the intellectual disability and neuromuscular findings, the results of brain imaging, electromyography and muscle biopsy appeared normal in the initial erlin2-deficient patients."
63,ALL,26,"Another family was found to have a homozygous null mutation in ERLIN2, with affected individuals presenting with a hereditary spastic paraplegia phenotype."
83,ALL,NA,"Clinical studies were designed to detail the phenotypic features of NGLY1-CDDG. Blood, urine, cerebral spinal fluid (CSF), lymphoblasts, and primary dermal fibroblasts were collected, analyzed, and stored. Studies included brain magnetic resonance imaging and spectroscopy (MRI and MRS, supplementary methods), routine and overnight electroencephalograms (EEGs) with a limited montage performed during a sleep study, electromyogram (EMG, supplementary methods) and nerve conduction studies (NCS, supplementary methods), indirect calorimetry, awake and sedated eye examination with Schirmer II testing, optical coherence tomography scans and electroretinography, behavioral determination of pure tone thresholds, tympanometry, distortion product otoacoustic emissions, auditory brainstem evoked potentials (ABR), quantitative sweat analysis autonomic testing (QSWEAT, supplementary methods), gastric aspiration, swallow study, skeletal survey, bone age, dual X-ray absorptiometry (DEXA), abdominal ultrasound, vibration controlled transient elastography (Fibroscan)12, echocardiogram, and electrocardiogram. Consultations included clinical neurology, audiology, nutrition, ophthalmology, hepatology, growth, puberty and hormonal studies, allergy and immunology, genetic counseling, physiatry, and speech, occupational, and physical therapy."
84,ALL,NA,"Eleven individuals underwent developmental psychological evaluations, consisting of at least the Vineland Adaptive Behavior Scales, 2nd edition. Cognitive function was assessed with testing specific for age and developmental level that provided either an intelligence quotient (IQ) or developmental quotient (DQ) score."
85,ALL,NA,"The Nijmegen pediatric CDG rating scale, a measure of clinical disease progression developed for CDG, was applied to all affected individuals younger than 18 years."
131,ALL,NA,"Compared to the reference population, N-acetylaspartylglutamate + N-acetylaspartate (NAA) was lower than normal in the left centrum semiovale (LCSO) (p=0.004), the midline parietal grey matter (PGM) (p=0.02), and superior cerebellar vermis (SVERM) (p<0.0001). There was a deficit of glutamine + glutamate + gamma-aminobutyric acid (Glx) in the PGM (p=0.03), LCSO (p=0.01), and pons (p=0.0002). Choline was higher than expected for age only in the LCSO (p=0.0097), and myo-inositol was higher than expected for age in the pons (p=0.002). Multiple correlations between these MRS-measured metabolites and age, functional assessments, brain volume, and neurotransmitters in the CSF were found. The general trend showed that the differences noted above became more pronounced with increasing age, worsening function, and lower brain volume. MRS metabolite measurements did not correlate with total CSF protein, CSF albumin, or CSF/serum albumin ratio. There was a weak correlation (p=0.09) between atrophy and total CSF protein, but not CSF albumin or CSF/serum albumin ratio."
169,ALL,NA,"Strong correlation between brain atrophy on MRI and functional assessments suggests that loss of neurons contributes to the functional impairment. The atrophy also correlated with CSF metabolites (BH4, 5-HIAA, HVA), which are known to be lower when there is damage to neurotransmitter producing neurons. This suggests that these biochemical abnormalities may be secondary to brain atrophy."


In [25]:
patients = (
    pd.concat([
        patients_raw.pipe(lambda df: df[df['patient_id,external_study,details'].isnull()]).drop(columns='patient_id,external_study,details'),
        (
            patients_raw.dropna(subset='patient_id,external_study,details')
            ['patient_id,external_study,details'].str.split(',', n=2, expand=True)
            .rename(columns={0: 'patient_id', 1: 'external_study', 2: 'details'})
        )
    ], ignore_index=True)
)
assert len(patients_raw) == len(patients)
path = utils.get_paths().output_data / "patients_1.1.tsv"
if not path.exists():
    patients.to_csv(path, sep="\t", index=False)
patients.info()
patients.head(15)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 180 entries, 0 to 179
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   patient_id      180 non-null    object
 1   external_study  21 non-null     object
 2   details         180 non-null    object
 3   doc_id          173 non-null    object
 4   doc_filename    173 non-null    object
dtypes: object(5)
memory usage: 7.2+ KB


,patient_id,external_study,details,doc_id,doc_filename
0,ALL,NaN,"All patients had global developmental delay, a movement disorder, and hypotonia.",PMC4243708,PMC4243708.txt
1,SOME,NaN,7 out of 8 patients had hypo- or alacrima.,PMC4243708,PMC4243708.txt
2,SOME,NaN,6 out of 7 patients had elevated liver transaminases.,PMC4243708,PMC4243708.txt
3,SOME,NaN,6 out of 8 patients had microcephaly.,PMC4243708,PMC4243708.txt
4,SOME,NaN,6 out of 8 patients had diminished reflexes.,PMC4243708,PMC4243708.txt
5,SOME,NaN,5 out of 6 patients had hepatocyte cytoplasmic storage material or vacuolization.,PMC4243708,PMC4243708.txt
6,SOME,NaN,4 out of 8 patients had seizures.,PMC4243708,PMC4243708.txt
7,ALL,NaN,The most common deleterious allele was the nonsense mutation c.1201A>T (p.R401X).,PMC4243708,PMC4243708.txt
8,ALL,NaN,"NGLY1 deficiency is a novel autosomal recessive disorder of the ERAD pathway associated with neurological dysfunction, abnormal tear production, and liver disease. The majority of patients detected to date carry a specific nonsense mutation that appears to be associated with severe disease.",PMC4243708,PMC4243708.txt
9,Patient 1,Need et al.,"3-year-old boy with compound heterozygous inactivating mutations in NGLY1. Clinical phenotype suggestive of a congenital disorder of glycosylation, although repeated transferrin isoelectric focusing and N-glycan analyses were normal. Liver biopsy showed accumulation of an amorphous unidentified substance throughout the cytoplasm, a finding likely consistent with NGLY1 dysfunction, which would be expected to result in abnormal accumulation of misfolded glycoproteins because of impaired cytosolic degradation.",PMC4243708,PMC4243708.txt


#### Analysis

In [30]:
(
    patients[['doc_id', 'patient_id']]
    .value_counts()
    .reset_index()
)

,doc_id,patient_id,count
0,PMC7477955,ALL,55
1,PMC7477955,SOME,13
2,PMC4243708,SOME,13
3,PMC4243708,ALL,12
4,PMC4243708,Patient 3,7
5,PMC4243708,Patient 6,6
6,PMC4243708,Patient 4,5
7,PMC4243708,Patient 5,5
8,PMC4243708,Patient 7,5
9,PMC4243708,Patient 2,5


In [32]:
(
    patients
    .pipe(lambda df: df[~df['patient_id'].isin(['ALL', 'SOME', '---'])])
)

,patient_id,external_study,details,doc_id,doc_filename
9,Patient 1,Need et al.,"3-year-old boy with compound heterozygous inactivating mutations in NGLY1. Clinical phenotype suggestive of a congenital disorder of glycosylation, although repeated transferrin isoelectric focusing and N-glycan analyses were normal. Liver biopsy showed accumulation of an amorphous unidentified substance throughout the cytoplasm, a finding likely consistent with NGLY1 dysfunction, which would be expected to result in abnormal accumulation of misfolded glycoproteins because of impaired cytosolic degradation.",PMC4243708,PMC4243708.txt
11,Patient 1,Duke University,Exome sequencing was performed using the Illumina HiSeq2000 platform and the Agilent SureSelect Human All Exon 50 Mb Kit.,PMC4243708,PMC4243708.txt
12,Patient 5,University of British Columbia,Exome sequencing was performed using the Illumina HiSeq2000 platform and the Agilent SureSelect Human All Exon 50 Mb Kit.,PMC4243708,PMC4243708.txt
13,Patient 6,University of British Columbia,Exome sequencing was performed using the Illumina HiSeq2000 platform and the Agilent SureSelect Human All Exon 50 Mb Kit.,PMC4243708,PMC4243708.txt
14,Patient 3,Stanford University,The patient and parents were sequenced using both Illumina HiSeq2000 and Complete Genomics platforms. Variants in Illumina-sequenced reads were called using both the Hugeseq and Real Time Genomics pipelines and Complete Genomics variants were identified by their own variant callers.,PMC4243708,PMC4243708.txt
15,Patient 3,Baylor College of Medicine,DNA was capture- sequenced using a commercially developed capture reagent (VCRome2). Sequence data were generated on an Illumina HiSeq2000 producing an average coverage of 80× with >90% of targeted bases at 20× coverage or higher.,PMC4243708,PMC4243708.txt
16,Patient 2,"Baylor College of Medicine Whole Genome Laboratory, Houston, Texas",WES was performed on a clinical basis.,PMC4243708,PMC4243708.txt
17,Patient 6,"Emory Genetics Laboratory, Atlanta, Georgia",WES was performed on a clinical basis.,PMC4243708,PMC4243708.txt
18,Patient 7,"Emory Genetics Laboratory, Atlanta, Georgia",WES was performed on a clinical basis.,PMC4243708,PMC4243708.txt
19,Patient 4,Duke University,"Sanger sequencing of NGLY1 was performed and results were confirmed by a clinical laboratory (GeneDx, Gaithersburg, Maryland).",PMC4243708,PMC4243708.txt


In [4]:
text = """
Neurologic Phenotype
Seven of twelve subjects had clinical seizures, and one had subclinical seizures recognized on previous EEG. Details regarding age of onset, seizure type and frequency, medications, and EEG findings are noted in Supplementary Table S. On overnight EEG, only one individual (#6) had active seizures recorded, but seven had multifocal epileptiform activity. There were no age or genotype differences between individuals having seizures and those without. In fact, in each sibling pair, one had seizures and the other did not.
All twelve individuals exhibited hyperkinetic movement disorders that included choreiform, athetoid, dystonic, myoclonic, action tremor, and dysmetric movements and were more severe in the younger individuals (Supplementary Movie S1).

Brain MRI and MRS
Eleven individuals underwent MRI and MRS of the brain. Clinical assessment of the images was not striking (Figure 3). Delayed myelination was present in three of the four youngest individuals, but all the older individuals had complete myelination. Six of nine individuals had qualitatively-evident cerebral atrophy that ranged from slight to moderate. Four individuals (#1, #2, #6, #10) also had slight cerebellar atrophy. The atrophy tended to be greater in the older individuals (p=0.17, Supplementary Figure S3), and in one teenager (#11) follow-up imaging showed atrophy measurably worse after a 20-month interval (net loss of 34 cm3 relative to expected). Increased atrophy correlated with worsening of all functional measurements (Supplementary Figure S3), including IQ or DQ (p<0.03), Vineland assessments (p<0.03), and Nijmegen scores (p=0.01). Brain volume also directly correlated with CSF levels of 5-HIAA (p=0.03), tetrahydrobiopterin (p=0.02), and 5-HVA (p=0.06) (Supplementary Figure S3).
"""

In [10]:
text = """
Table 1
Clinical and molecular findings in NGLY1 deficiency
Patient 1	Patient 2	Patient 3	Patient 4	Patient 5	Patient 6	Patient 7	Patient 8	Totals
Age	5 y	20 y	4 y	2 y	d.5 y	d.9 m	3 y	16 y	
Gender	M	F	F	M	M	F	F	F	
Ethnicity	Caucasian	Caucasian	Caucasian	Caucasian	Caucasian	Caucasian	Caucasian	Caucasian	
Consanguinity	−	+	−	−	−	−	−	−	1/8
Mutations (maternal/paternal allele)	c.C1891del (p.Q631S)/c.1201A>T(p.R401X)	c.1370dupG(p.R458fs)/c.1370dupG(p.R458fs)	c.1205_1207del(p.402_403del)/c.1570C>(p.R524X)	c.1201A>T(p.R401X)c.1201A>T(pR401X)	c.1201A>T(p.R401X)/c.1201A>T(p.R401X)	c.1201A>T(p.R401X)/c.1201A>T(p.R401X)	c.1201A>Y(p.R401X)/c.1201A>T(p.R401X)	c1201A>T(p.R401X)/c.1201A>T(p.R401X)	
IUGR	−	+	−	+	+	+	−	+	5/8
Brain imaging abnormalities	+a	−b	+c	+d	+e	+f	−	+g	6/8
Global developmental delay	+	+	+	+	+	+	+	+	8/8
Microcephalyh	−	+	+	−	+	+	+	+	6/8
Hypotonia	+	+	+	+	+	+	+	+	8/8
Movement disorder	+	+	+	+	+	+	+	+	8/8
EEG abnormalities	+	+	+	+	+	+	−	+	7/8
↓DTRs	+	+	−	+	+	−	+	+	6/8
Seizures	+	−	−	+	+	−	−	+	4/8
Ocular apraxia	−	+	+	−	−	−	+	+	4/8
Alacrima/hypolacrima	+	+	+	+	+	−	+	+	7/8
Corneal ulcerations/scarring	+	+	−	+	−	−	−	+	4/8
Chalazions	+	−	+	+	−	−	+	−	4/8
Strabismus	−	−	+	+	−	−	+	+	5/8
ABR abnormalities	−	−	+	+	−	ND	ND	ND	2/5
Lactic acidosis	−	+	+	+	−i	ND	+	ND	4/6
Neonatal jaundice	+	−	+	+	−	−	+	−	4/8
Elevated liver transaminases	+	+	+	+	+	ND	+	−	6/7
Elevated AFP	+	−	−j	+	+	ND	ND	ND	3/5
Liver fibrosis	+	−	−	+	−	−	ND	ND	2/6
Liver storage or vacuolization	+	+	+	−	+k	+l	ND	ND	5/6
Constipation	+	+	+	+	+	−	+	+	7/8
Dysmorphic features	−	−	−	−	+m	+n	+o	+p	4/8
Scoliosis	−	+	−	+	+	−	−	+	4/8
Small hands/feet	+	−	+	+	−	−	−	+	4/8
Peripheral neuropathyq	+	+	ND	ND	+	ND	ND
"""

In [12]:
text = """
Patient 2, a now 20-year-old female, was born at 39 weeks of gestation via Cesarean section because of intrauterine growth retardation and an abnormal appearing placenta. At four months of age, hypotonia, developmental delay and elevated liver transaminases were noted. At approximately 4 years of age, a slight intention tremor and frequent involuntary movements of her neck, hands and arm were observed. At 5 years of age, she was noted to have ocular apraxia, distal tapering of hands and feet, and diminished deep tendon reflexes. She has cortical vision impairment, as well as alacrima and dry eyes that require lubrication, but has not developed corneal scarring. Presently, she has marked intellectual disabilities and requires total care. She has very little expressive speech and communicates through an electronic speech-generating device. She continues to ambulate with a walker.
"""

In [13]:
response = llm.chat_completion("patient_extraction_1.txt", model="gpt-4", temperature=0.0, disease=utils.NGLY1_DEFICIENCY, text=text)
print(response)

INFO:ngly1_gpt.llm:Prompt (temperature=0.0, model=gpt-4):
Text will be provided that contains information from a published, biomedical research article about NGLY1 deficiency.  Extract details about the patients discussed in this text: 

--- BEGIN TEXT ---

Patient 2, a now 20-year-old female, was born at 39 weeks of gestation via Cesarean section because of intrauterine growth retardation and an abnormal appearing placenta. At four months of age, hypotonia, developmental delay and elevated liver transaminases were noted. At approximately 4 years of age, a slight intention tremor and frequent involuntary movements of her neck, hands and arm were observed. At 5 years of age, she was noted to have ocular apraxia, distal tapering of hands and feet, and diminished deep tendon reflexes. She has cortical vision impairment, as well as alacrima and dry eyes that require lubrication, but has not developed corneal scarring. Presently, she has marked intellectual disabilities and requires total c

INFO:ngly1_gpt.llm:Response:
patient_id|external_study|details
Patient 2|NA|20-year-old female, born at 39 weeks of gestation via Cesarean section due to intrauterine growth retardation and an abnormal appearing placenta. At four months of age, hypotonia, developmental delay and elevated liver transaminases were noted. At approximately 4 years of age, a slight intention tremor and frequent involuntary movements of her neck, hands and arm were observed. At 5 years of age, she was noted to have ocular apraxia, distal tapering of hands and feet, and diminished deep tendon reflexes. She has cortical vision impairment, as well as alacrima and dry eyes that require lubrication, but has not developed corneal scarring. Presently, she has marked intellectual disabilities and requires total care. She has very little expressive speech and communicates through an electronic speech-generating device. She continues to ambulate with a walker.
patient_id|external_study|details
Patient 2|NA|20-year-old

In [9]:
print(response)

patient_id|external_study|details
ALL|NA|Seven of twelve subjects had clinical seizures, and one had subclinical seizures recognized on previous EEG. On overnight EEG, only one individual had active seizures recorded, but seven had multifocal epileptiform activity. There were no age or genotype differences between individuals having seizures and those without. In each sibling pair, one had seizures and the other did not.
ALL|NA|All twelve individuals exhibited hyperkinetic movement disorders that included choreiform, athetoid, dystonic, myoclonic, action tremor, and dysmetric movements and were more severe in the younger individuals.
ALL|NA|Eleven individuals underwent MRI and MRS of the brain. Delayed myelination was present in three of the four youngest individuals, but all the older individuals had complete myelination. Six of nine individuals had qualitatively-evident cerebral atrophy that ranged from slight to moderate.
#1|NA|Had slight cerebellar atrophy.
#2|NA|Had slight cerebel